<img src='NEFUlogo.png' align='left'>

# Fire up Graphlab, re, pandas, string, collections

In [1]:
import graphlab
import re
import pandas
import string

In [2]:
from collections import OrderedDict
from collections import Counter

## 1. read data using graphlab.SFrame.read_csv

In [3]:
data = graphlab.SFrame.read_csv('SogouM.txt', delimiter='\t', header=True, column_type_hints=[str, str, str, str, str])

This non-commercial license of GraphLab Create is assigned to guoxiuhe@nefu.edu.cn and will expire on April 02, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-03 09:30:21,197 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: C:\Users\heguoxiu\AppData\Local\Temp\graphlab_server_1462239017.log.0


Finished parsing file E:\Python\2GraduationProject\SogouM.txt

Parsing completed. Parsed 10000 lines in 0.101006 secs.

## 2. read and process the ODP data

In [4]:
#odp = graphlab.SFrame.read_csv('ODP.csv', delimiter='\t', header=False)

In [5]:
#odp.show()

In [6]:
#odp['X1'] = odp['X1'].apply(lambda str1: re.match('(.*?)" class="listinglink', str1).group(1))

In [7]:
#odp.save('ODP.csv')

In [8]:
#odp = odp.unique()

## separate the data['result_click'] into data['result_and_click']

In [9]:
data['result_and_click'] = data['result_click'].apply(lambda x : re.split(r'\s+',x))

In [10]:
def getfirst(l):
    return int(l[0])
def getsecond(l):
    return int(l[1])

data['result_rank'] = data['result_and_click'].apply(lambda l: getfirst(l))
data['click_rank'] = data['result_and_click'].apply(lambda l: getsecond(l))

## remove the [] from data['query']

In [11]:
data['query'] = data['query'].apply(lambda x: str(x)).apply(lambda x: x.strip().lstrip('[').rstrip(']'))

## preprocessing of url(data['url']), get the data['url_split'] and data['word_count']

In [13]:
data['url_split'] = data['url'].apply(lambda x: str(x)).apply(lambda x: x.lower().replace('\"', '').replace('.shtml', '').replace('.htm', '').replace('.com', '').replace('.cn', '').replace('.html', '').replace('.org', '').replace('.net', '').strip().lstrip('[').rstrip(']')).apply(lambda x: re.split(r'/',x))

In [14]:
data['word_count'] = graphlab.text_analytics.count_words(data['url_split'])

## remove the tail of the url

In [15]:
def remove_tail(url):
    if url[-1] == '/':
        return url[:-1]
    else:
        return url[:]

In [16]:
data['url'] = data['url'].apply(remove_tail)

In [19]:
data.save('Sogou.csv')

In [14]:
#data['query'].save('query.csv')

In [ ]:
from collections import OrderedDict
from collections import Counter
word_counter = Counter()
for each in data['word_count']:
    word_counter = word_counter + Counter(each)
word_dict = OrderedDict(word_counter)

In [11]:
from collections import OrderedDict
from collections import Counter
from multiprocessing.dummy import Pool
pool = Pool(10)
def add_Counter(each):
    global word_counter
    word_counter = word_counter + Counter(each)
word_counter = Counter()
results = pool.map(add_Counter, data['word_count'])
word_dict = OrderedDict(word_counter)
pool.close()
pool.join()

In [20]:
#OrderedDict(sorted(word_dict.items(), key=lambda t: t[1]))

# Overview the Url_counter

In [24]:
url_counter = Counter()
for url in data['url']:
    url_counter[url] = url_counter[url] + 1

In [25]:
url_counter = sorted(url_counter.items(), key=lambda t: t[1], reverse= True)

In [27]:
f = open('url_counter.csv', 'a')

In [30]:
for (k, v) in url_counter:
    f.writelines(k+','+str(v)+'\n')
f.close()